**OBJECTIVE**
The objective of this analysis is to create a pricing strategy for the company XYZ using the available datasets.The datasets used in this data analysis are Historical sales,Ads performance,Returns data,Competitor data,Pricing data and Inventory health.The goal is to create a more systematic, data-driven pricing approach that can adapt to changing conditions and support better operational decisions.

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [ ]:
sales=pd.read_csv("Historical_Sales.csv")
sales.head()
sales.info()
sales.describe()

In [ ]:
sales.columns = (
    sales.columns
    .str.strip()
    .str.lower()
    .str.replace(r'[\s\-]+', '_', regex=True)
)
sales.columns = sales.columns.str.replace(r'_+', '_', regex=True)


sales.info()


In [ ]:
#Change the format of the date column and remove null values
sales['date'] = pd.to_datetime(sales['date'], errors='coerce').dt.date



In [ ]:
#remove nulls from all the columns
sales=sales.dropna(subset=['date'])
sales=sales.dropna(subset=['sku'])







In [ ]:
sales['sessions_total'] = sales['sessions_total'].fillna(0)
sales['page_views_total'] = sales['page_views_total'].fillna(0)
sales['units_ordered'] = sales['units_ordered'].fillna(0)
sales['ordered_product_sales'] = sales['ordered_product_sales'].fillna(0)
sales['total_order_items'] = sales['total_order_items'].fillna(0)


In [ ]:
ads=pd.read_csv('Ads_Performance.csv')
ads.head()
ads.info()
ads.describe()


In [ ]:
ads.columns = (
    ads.columns
    .str.strip()
    .str.lower()
    .str.replace(r'[\s\-]+', '_', regex=True)
    .str.replace(r'_+', '_', regex=True)
)
ads.columns.tolist()


In [ ]:
ads['date'] = pd.to_datetime(ads['date'], errors='coerce').dt.date
ads['date'].isna().sum()
ads = ads.dropna(subset=['date'])


In [ ]:
ads.isna().sum()

In [ ]:
rate_cols = [
    'costperclick',
    'clickthroughrate',
    'acosclicks7d','acosclicks14d','roasclicks7d','roasclicks14d'
]

ads[rate_cols] = ads[rate_cols].fillna(0)
ads.isna().sum()


In [ ]:
competitor=pd.read_csv('Competitor_Data.csv')
competitor.head()
competitor.info()
competitor.describe()



In [ ]:
competitor.columns = (
    competitor.columns
    .str.strip()
    .str.lower()
    .str.replace(r'[\s\-]+', '_', regex=True)
    .str.replace(r'_+', '_', regex=True)
)
competitor.columns.tolist()

In [ ]:
competitor['avg_competitor_price'] = (
    competitor['avg_competitor_price']
    .replace(r'[\$,]', '', regex=True)
    .astype(float)
)
competitor['lowest_competitor_price'] = (
    competitor['lowest_competitor_price']
    .replace(r'[\$,]', '', regex=True)
    .astype(float)
)
competitor.head()

competitor['highest_competitor_price'] = competitor['highest_competitor_price'].replace(
    r'[\$,-]+', '', regex=True
)
competitor['highest_competitor_price'] = pd.to_numeric(
    competitor['highest_competitor_price'],
    errors='coerce'
)
competitor.isna().sum()






In [ ]:
competitor=competitor.dropna(subset=['avg_competitor_price'])
competitor=competitor.dropna(subset=['highest_competitor_price'])
competitor.isna().sum()
competitor.info()

In [ ]:
inventory=pd.read_csv('Inventory_Health.csv')
inventory.head()
inventory.info()
inventory.describe()

In [ ]:
inventory.columns = (
    inventory.columns
    .str.strip()
    .str.lower()
    .str.replace(r'[\s\-]+', '_', regex=True)
    .str.replace(r'_+', '_', regex=True)
)
inventory.info()


In [ ]:
inventory.isna().sum()

In [ ]:
inventory['total_inventory'] = (
    inventory['available']
    + inventory['reserved_quantity']
    + inventory['unfulfillable_quantity']
)
inventory.isna().sum()

In [ ]:
returns=pd.read_csv('Returns_Data.csv')
returns.head()
returns.info()

In [ ]:
returns.columns = (
    returns.columns
    .str.strip()
    .str.lower()
    .str.replace(r'[\s\-\()]+', '_', regex=True)
    .str.replace(r'_+', '_', regex=True)
)

returns.columns.tolist()


In [ ]:
returns['return_quantity_last_7_days_'] = pd.to_numeric(
    returns['return_quantity_last_7_days_'],
    errors='coerce'
)
returns.info()
returns['return_quantity_last_7_days_']=returns['return_quantity_last_7_days_'].fillna(0)
returns.isna().sum()


In [ ]:
pricing=pd.read_csv('Pricing_Data.csv')
pricing.head()
pricing.info()
pricing.describe()

In [ ]:
pricing.columns = (
    pricing.columns
    .str.strip()
    .str.lower()
    .str.replace(r'[\s\-\%]+', '_', regex=True)
    .str.replace(r'_+', '_', regex=True)
)

pricing.columns.tolist()

In [ ]:
pricing['fba_fee'] = pricing['fba_fee'].replace(
    r'[\$,-]+', '', regex=True
)
pricing['fba_fee'] = pd.to_numeric(
    pricing['fba_fee'],
    errors='coerce'
)
pricing['storage_fee'] = pricing['storage_fee'].replace(
    r'[\$,-]+', '', regex=True)

pricing['storage_fee'] = pd.to_numeric(
    pricing['storage_fee'],
    errors='coerce'
)

pricing['handling_cost'] = pricing['handling_cost'].replace(
    r'[\$,-]+', '', regex=True)

pricing['handling_cost'] = pd.to_numeric(
    pricing['handling_cost'],
    errors='coerce'
)

pricing['cost'] = pricing['cost'].replace(
    r'[\$,-]+', '', regex=True)

pricing['cost'] = pd.to_numeric(
    pricing['cost'],
    errors='coerce'
)

pricing['current_price'] = pricing['current_price'].replace(
    r'[\$,-]+', '', regex=True)

pricing['current_price'] = pd.to_numeric(
    pricing['current_price'],
    errors='coerce'
)



In [ ]:
pricing['minimum_acceptable_margin_'] = (
    pricing['minimum_acceptable_margin_']
    .astype(str)
    .str.replace('%', '', regex=False)
    .str.strip()
)
pricing['minimum_acceptable_margin_'] = pd.to_numeric(
    pricing['minimum_acceptable_margin_'],
    errors='coerce'
)

pricing['minimum_acceptable_margin_'] = (
    pricing['minimum_acceptable_margin_'] / 100
)




In [ ]:
pricing['target_gross_margin_'] = pricing['target_gross_margin_'].replace(
    r'[\%,-]+', '', regex=True
)
pricing['target_gross_margin_'] = pd.to_numeric(
    pricing['target_gross_margin_'],
    errors='coerce'
)

In [ ]:
pricing['handling_cost']=pricing['handling_cost'].fillna(0)
pricing.isna().sum()

In [ ]:
sales.info()

In [ ]:
#Grouping and aggregating the sales table as per SKU
sales_sku = (
    sales
    .groupby('sku', as_index=False)
    .agg(
        total_units_sold=('units_ordered', 'sum'),
        total_revenue=('ordered_product_sales', 'sum'),
        total_order_items=('total_order_items', 'sum'),
        total_sessions=('sessions_total', 'sum'),
        total_page_views=('page_views_total', 'sum')
    )
)


In [ ]:
#Create required metrics for the sales table
#1.Average selling price
sales_sku['avg_selling_price'] = (
    sales_sku['total_revenue'] / sales_sku['total_units_sold']
).replace([np.inf, -np.inf], 0)

#2.Conversion rate
sales_sku['conversion_rate']=(sales_sku['total_units_sold']/sales_sku['total_sessions']).replace([np.inf,-np.inf],0)

#3.Average daily units sold
total_days= sales['date'].nunique()
sales_sku['avg_daily_units_sold'] = (
   sales_sku['total_units_sold'] / total_days
)

#4.Revenue share
sales_sku['revenue_share'] = (
    sales_sku['total_revenue'] / sales_sku['total_revenue'].sum()
)

#5.Rolling velocity(Sales for 30 days)
daily_sales = (
    sales
    .groupby(['sku', 'date'], as_index=False)
    .agg(
        units_sold=('units_ordered', 'sum')
    )
)
daily_sales = daily_sales.sort_values(['sku', 'date'])

daily_sales['sales_velocity_30d'] = (
    daily_sales
    .groupby('sku')['units_sold']
    .rolling(window=30, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

velocity_sku = (
    daily_sales
    .groupby('sku', as_index=False)
    .tail(1)[['sku', 'sales_velocity_30d']]
)

sales_sku = sales_sku.drop(
    columns=['sales_velocity_30d_x', 'sales_velocity_30d_y'],
    errors='ignore'
)


velocity_sku.head()
velocity_sku['sku'].is_unique

sales_sku = sales_sku.drop(
    columns=['sales_velocity_30d', 'sales_velocity_30d_x', 'sales_velocity_30d_y'],
    errors='ignore'
)
sales_sku = sales_sku.merge(
    velocity_sku,
    on='sku',
    how='left'
)
sales_sku[['sku', 'sales_velocity_30d']].head()
sales_sku['sales_velocity_30d'].isna().sum()




















In [ ]:
ads['sku'].is_unique

ads_sku = (
    ads
    .groupby('sku', as_index=False)
    .agg(
        total_impressions=('impressions', 'sum'),
        total_clicks=('clicks', 'sum'),
        total_ad_spend=('spend', 'sum'),
        ad_purchases_30d=('purchases30d', 'sum'),
        ad_units_sold_30d=('unitssoldclicks30d', 'sum'),
        ad_sales_30d=('sales30d', 'sum')
    )
)

ads.info()






In [ ]:
#Create required metrics on ads performance dataset

#1.ROAS
ads_sku['roas'] = (
    ads_sku['ad_sales_30d'] / ads_sku['total_ad_spend']
).replace([np.inf, -np.inf], 0)

#2.CTR
ads_sku['ctr'] = (
    ads_sku['total_clicks'] / ads_sku['total_impressions']
).replace([np.inf, -np.inf], 0)



In [ ]:
#creating metrics on pricing dataset
#1.Adding total unit cost
pricing['total_unit_cost'] = pricing['cost'] + pricing['handling_cost']

pricing['target_gross_margin_']=pricing['target_gross_margin_']/100
#pricing = pricing.drop(columns=['target_gross_margin'])


pricing.isna().sum()





In [ ]:
returns['sku'].is_unique
returns.isna().sum()
returns.head()

In [ ]:
#merge the datasets into one master table
master=sales_sku.copy()
master=master.merge(inventory,on='sku',how='left')
master = master.merge(ads_sku,on='sku',how='left')
pricing_sku=pricing.copy()
master=master.merge(pricing_sku,on='sku',how='left')
competitor_sku=competitor.copy()
master=master.merge(competitor_sku,on='sku',how='left')
master=master.merge(returns,on='sku',how='left')
#master.head()
master['competitor_count'] = master['competitor_count'].fillna(0)
master.isna().sum().sort_values(ascending=False)







In [ ]:
master['sku'].is_unique
master.shape
#master.isna().sum()[master.isna().sum() > 0]


In [ ]:
#Create metrics on the merged master data
#1.Inventory
master['days_of_inventory_cover'] = (
    master['total_inventory'] / master['avg_daily_units_sold']
).replace([np.inf, -np.inf], 0)

#2 Rate of return
master['return_rate_30d'] = (
    master['return_quantity_last_30_days_'] / master['total_units_sold']
).replace([np.inf, -np.inf], 0)

#3 Current margin
master['current_margin_pct'] = (
    (master['current_price'] - master['total_unit_cost']) /
    master['current_price']
)

#4. Competitor price
master['price_gap_pct'] = (
    (master['current_price'] - master['avg_competitor_price']) /
    master['avg_competitor_price']
)

#when competitor price is missing
master.loc[
    master['avg_competitor_price'].isna(),
    'price_gap_pct'
] = 0



master['avg_competitor_price'] = master['avg_competitor_price'].replace(0, np.nan)
master['price_gap_pct'] = (
    (master['current_price'] - master['avg_competitor_price']) /
    master['avg_competitor_price']
)
master.loc[
    master['avg_competitor_price'].isna(),
    'price_gap_pct'
] = 0

master[['days_of_inventory_cover',
        'return_rate_30d',
        'price_gap_pct',
        'current_margin_pct']].describe()









In [ ]:
#removing unwanted columns from master dataset
master=master.drop(columns=['condition','currency','volume_unit_measurement','storage_type','marketplace','country_of_origin'],errors='ignore')
master.head()
#master['sku'].is_unique


In [ ]:
#Visualizations
plt.figure()
plt.scatter(master['current_price'],master['avg_daily_units_sold'],alpha=0.7)
plt.xlabel('Current Price')
plt.ylabel('Avg Daily Units Sold')
plt.title('Price vs Average units sold')

plt.show()


The above chart shows the relationship between Current Price and Average daily units sold for each SKU.Sales volume is high for low to moderate prices.Higher priced products tend to sell in less volume.This shows price sensitivity to the number of units sold.

In [ ]:
plt.figure()
plt.scatter(master['sales_velocity_30d'],master['days_of_inventory_cover'],alpha=0.7)
plt.xlabel('Sales_velocity_30d')
plt.ylabel('Days_of_inventory_cover')
plt.title('Sales_velocity_30d vs Days_of_inventory_cover')
plt.show()

The above plot shows the relationship between sales velocity and days of inventory cover.Higher sales velocity is associated with lower days of inventory cover which implies that with higher velocity of sales, the product will have only less days of inventory cover.Slow moving products tend to accumulate excess inventory.

In [ ]:
plt.figure()
plt.scatter(master['current_price'],master['avg_competitor_price'],alpha=0.7)
plt.xlabel('current_price')
plt.ylabel('avg_competitor_price')
plt.title('Current price vs Avg competitor price')
plt.show()

The above plot shows the relationship between Current price and average competitor price. The relationship is almost a linear relationship which implies that current prices closely track average competitor prices.Current prices are market aligned pricing as per the plot.

In [ ]:
plt.figure()
plt.scatter(master['return_rate_30d'],master['total_units_sold'],alpha=0.7)
plt.xlabel('return_rate')
plt.ylabel('no_of_units_sold')
plt.title('Return rate vs no of units sold')
plt.show()

The above plot shows the relationship between return rate and number of units sold. The plot implies that products with lower return rate shows higher sales while products with higher return rate shows lower sales.

In [ ]:
plt.figure()
plt.scatter(master['roas'],master['price_gap_pct'],alpha=0.7)
plt.xlabel('ROAS')
plt.ylabel('price gap')
plt.title('ROAS vs Price gap')
plt.show()

The above plot shows the relationship between ROAS and price gap. Ads perform better when the prices are below market level. And higher priced products tend to show lower ad efficiency.

In [ ]:
master.head()

In [ ]:

master['target_gross_margin_'] = master['target_gross_margin_'] *1000
master[['target_gross_margin_', 'current_margin_pct']].describe()



In [ ]:
plt.figure()
plt.scatter(master['sales_velocity_30d'],master['price_gap_pct'],alpha=0.7)
plt.xlabel('Sales Velocity 30 days')
plt.ylabel('Price gap')
plt.title('Sales Velocity vs Price gap')
plt.show()

The above plot shows the relationship between Sales velocity and price gap.Products which are priced above the market level tends to sell slowly while products which are priced below the market prices tends to have more sales velocity.

#Deciding the Pricing Strategy Framework
The main components that affect the pricing logic are:
1.Demand (Sales velocity,Average units sold)
2.Inventory pressure(Days of inventory cover)
3.Market(Competitor price gap)
4.Advertising efficiency
5.Return rate which implies quality
6.Margin

**Demand:** High Demand shows that the product is fast moving. Customers are

ready to buy the product even if the price is slightly increased. There is a possibility to increase price when demand is high.Whereas low demand shows that the customers are not interested in buying the product and the price could not be further increased. The price should be decreased in this case.

**Inventory pressure:** High days of inventory cover suggests that the product is more in stock. In this case,it is essential to increase the sales. The price can be decreased in this case.Whereas low inventory cover suggests that the product is moving quickly and the price of the product can be increased or can be put on hold in this case.

**Market level/Competitor prices** : If the price gap between the product's current price and average market price is negative, there is a room to increase the price. Whereas if the price gap is positive,the product will be considered more expensive than the competitors and the price should be decreased in this case.

**Advertising Efficiency**: Higher return on ad spend shows that the customers are ready to buy the product in the current price and there is room for price increase. Whereas, if the ROAS is low,it means that the price should be reduced to attract more customers.

**Return Rate**: If the return rate for the past 30 days is higher, it shows that there is a quality issue in the product.In this case,increasing price will be riskier.If the return rate is lower,it shows that customers are satisfied with the product and in this case,price can be increased.

**Current Margin:**: If the current margin/profitability is lower, it means that the price of the product cannot be reduced further.Whereas if its high, we can decrease the price.



In [ ]:
#Set all the pricing action to Hold

master['pricing_action'] = 'Hold'


In [ ]:
#Pricing Logic: To increase the Price

master.loc[
    (master['days_of_inventory_cover'] < 30) &
    (master['sales_velocity_30d'] > master['avg_daily_units_sold']) &
    (master['price_gap_pct'] <= 0) &
    (master['roas'] > 2.5) &
    (master['return_rate_30d'] < 0.07),
    'pricing_action'
] = 'Increase Price'


In [ ]:
#To Decrease the Price
#decrease_mask=(
#    (master['days_of_inventory_cover'] > 45) &
#    (master['sales_velocity_30d'] < master['avg_daily_units_sold']) &
#    (master['price_gap_pct'] >= -0.05)
#)

#decrease_mask.sum()




In [ ]:
#To Decrease the Price
master.loc[
    (master['days_of_inventory_cover'] > 45) &
    (master['sales_velocity_30d'] < master['avg_daily_units_sold']) &
    (master['price_gap_pct'] >= -0.05),
    'pricing_action'
] = 'Decrease Price'

In [ ]:
#To keep the price in hold
master.loc[
    (master['pricing_action']=='Decrease Price')&
    (master['current_margin_pct'] < master['minimum_acceptable_margin_']),
    'pricing_action'
] = 'Hold'


In [ ]:
master['pricing_action'].value_counts()


The analysis shows that majority of the SKUs are held at current prices without any explicit increase/decrease in the price. Only a small set of products out of the 50 products recommends for price increase/decrease as per the pricing logic.

In [ ]:
summary=master[['sku','current_price','pricing_action','avg_competitor_price','price_gap_pct','sales_velocity_30d','avg_daily_units_sold','roas','days_of_inventory_cover','return_rate_30d','current_margin_pct']].sort_values('pricing_action')
summary